# Running a Parallel Tempering Bayeslands example

In this notebook, we will and run a Bayeslands model using the parallel tempering method.

## Bayes

Given some data, $D$, we make inference regarding unknown parameters, denoted by  $\theta$, via the posterior distribution $p(\theta|D)$,  

given by Bayes rule, $p(\theta|D)=\frac{p(D|\theta)p(\theta)}{P(D)}$

$p(D|\theta)$ is the likelihood of the data given the parameters

$p(\theta)$ is the prior

$p(D)$ is a normalizing constant and equal to $\int p(D|\theta)p(\theta)d\theta$. 

Bayeslands uses MCMC to obtain samples of $\theta$ from the posterior distribution by proposing values from some known distribution $q(\theta)$. The value of $\theta$ is  then set equal to these proposed values with a probability which ensures the detailed balance condition is met, otherwise the chain remains in its current position. The transition kernel which moves the Markov chain from one point to another in the parameter space is the product of the proposal distribution $q$ and the acceptance probability, $\alpha$. Under certain conditions, the draws from this transition kernel converge to draws from the stationary distribution, $p(\theta| D)$.

## Parallel Tempering

This method requires running Badlands thousands of times to explore the full parameter space. If a model takes 1 second, this is okay, but if a model takes a few minutes, then this very challenging. We use the concept of ***parallel tempering*** to help overcome this problem. Parallel tempering takes its name from thermodynamics of physical systems where the temperature of a system determines the ability to change. Parallel tempering is also known as replica exchange or the Multi-Markov Chain method. It is suitable for multi-modal distributions  by running multiple MCMC chains at different *temperatures* in parallel.

   Metropolis Hastings |   MH-Parallel Tempering
:-------------------------:|:-------------------------:
![](notebook_images/alg_MH.png)  |  ![](notebook_images/alg_MHPT.png)


The parallel tempering algorithm can be visualised as follows:

<img src="notebook_images/multi-core-ptbayeslands.png" width=500>

This comparison between the MCMC and Parallel Tempered exploration of rainfall in a Badlands model. Each sample is trying a slightly different value of rainfall (combined with other parametrs). In this case the ***true value*** of rainfall was 1.5 m/yr.

  Rainfall exploration with MCMC |   Rainfall exploration Parallel Tempering
:-------------------------:|:-------------------------:
![](notebook_images/rain_mcmc.png)  |  ![](notebook_images/rain_pt.png)





In [ ]:
#Set up the environemnt
%matplotlib inline
import sys
sys.path.append('scripts/')

#Import the script that produces a start and end topography
from ptBayeslands import *

# display plots in SVG format
#%config InlineBackend.figure_format = 'svg' 

In [ ]:
problemfolder = 'crater/'
xmlinput = problemfolder + 'crater.xml'
simtime = 100000
resolu_factor = 1
samples = 100

likelihood_sediment = True

#Set variables
#m = 0.5
m_min = 0.
m_max = 2.

#n = 1.
n_min = 0.
n_max = 2.

#rain_real = 1.5
rain_min = 0.
rain_max = 3.

#erod_real = 5.e-6
erod_min = 3.e-6
erod_max = 7.e-6

minlimits_vec=[rain_min,erod_min,m_min,n_min]
maxlimits_vec=[rain_max,erod_max,m_max,n_max]    


#you can have different ratio values for different parameters depending on the problem. 
stepsize_ratio  = 0.02 
stepratio_vec = [stepsize_ratio, stepsize_ratio, stepsize_ratio, stepsize_ratio] 


erodep_coords=np.array([[5,5],[10,10],[20,20],[30,30],[40,40],[50,50],[25,25],[37,30],[44,27],[46,10]])

In [ ]:
#update with additonal parameters. should have as many rows as parameters
true_parameter_vec = np.loadtxt(problemfolder + 'data/true_values.txt')
vec_parameters = np.random.uniform(minlimits_vec, maxlimits_vec) #  draw intial values for each of the free parameters
num_param = vec_parameters.size

In [ ]:
datapath = problemfolder + 'data/final_elev.txt'
groundtruth_elev = np.loadtxt(datapath)
groundtruth_erodep = np.loadtxt(problemfolder + 'data/final_erdp.txt')
groundtruth_erodep_pts = np.loadtxt(problemfolder + 'data/final_erdp_pts.txt')

fname = ""
run_nb = 0
while os.path.exists(problemfolder +'results_%s' % (run_nb)):
    run_nb += 1
if not os.path.exists(problemfolder +'results_%s' % (run_nb)):
    os.makedirs(problemfolder +'results_%s' % (run_nb))
    fname = (problemfolder +'results_%s' % (run_nb))
    
make_directory((fname + '/posterior/pos_parameters')) 
make_directory((fname + '/posterior/predicted_topo'))
make_directory((fname + '/posterior/pos_likelihood'))
make_directory((fname + '/posterior/accept_list'))
if likelihood_sediment==True:
    make_directory((fname + '/posterior/predicted_erodep'))

make_directory((fname + '/pred_plots'))
run_nb_str = 'results_' + str(run_nb)

In [ ]:
#-------------------------------------------------------------------------------------
# Number of chains of MCMC required to be run
# PT is a multicore implementation must num_chains >= 2
# Choose a value less than the numbe of core available (avoid context swtiching)
#-------------------------------------------------------------------------------------

num_chains = 4 
swap_ratio = 0.1 
burn_in =0.1 

#parameters for Parallel Tempering
maxtemp = int(num_chains * 5)/2

#how ofen you swap neighbours
swap_interval = int(swap_ratio * (samples/num_chains)) 

#How often output is created
num_successive_topo = 4
sim_interval = np.arange(0,  simtime+1, simtime/num_successive_topo) 

print("Max temp: ", maxtemp)
print("Samples: ", samples)
print("swap ratio: ",swap_ratio)
print("Number of chains: ", num_chains)
print("Swap interval: ", swap_interval)
print("Simulation time interval", sim_interval)

In [ ]:
timer_start = time.time()

#-------------------------------------------------------------------------------------
#Create A a Patratellel Tempring object instance 
#-------------------------------------------------------------------------------------
pt = ParallelTempering(  vec_parameters, num_chains, maxtemp, samples,swap_interval,fname, true_parameter_vec, num_param  ,  groundtruth_elev,  groundtruth_erodep_pts , erodep_coords, simtime, sim_interval, resolu_factor, run_nb_str, xmlinput,likelihood_sediment)

#-------------------------------------------------------------------------------------
# intialize the MCMC chains
#-------------------------------------------------------------------------------------
pt.initialize_chains( minlimits_vec, maxlimits_vec, stepratio_vec, likelihood_sediment,burn_in)

#-------------------------------------------------------------------------------------
#run the chains in a sequence in ascending order
#-------------------------------------------------------------------------------------
pos_param,likehood_rep, accept_list, combined_erodep, pred_elev = pt.run_chains()

#print('sucessfully sampled') 
timer_end = time.time() 



While we wait, explore the function that we a are running in the [script](scripts/ptBayeslands.py) we call.
As in the [Badlands Mountain Example](mountain.ipynb), you should be able to see where we manipulate the [XML file](crater/crater.xml) input variables. Look for the "model = badlandsModel()" line to see the familiar commands. 

In [ ]:
likelihood = likehood_rep[:,0] # just plot proposed likelihood  
likelihood = np.asarray(np.split(likelihood,  num_chains ))


plt.plot(likelihood.T)
plt.savefig( fname+'/likelihood.png')
plt.show()
plt.clf()
plt.plot(accept_list.T)
plt.savefig( fname+'/accept_list.png')
plt.show()
plt.clf()

for i in range(sim_interval.size):
        pos_ed  = combined_erodep[i, :, :] 

        #print(pos_ed) 
        erodep_mean = pos_ed.mean(axis=0)  
        erodep_std = pos_ed.std(axis=0)  
        plot_erodeposition(erodep_mean, erodep_std, groundtruth_erodep_pts[i,:], sim_interval[i], fname) 
        #np.savetxt(fname + '/posterior/predicted_erodep/com_erodep_'+str(sim_interval[i]) +'_.txt', pos_ed)

pred_erodep = np.zeros(( groundtruth_erodep_pts.shape[0], groundtruth_erodep_pts.shape[1] )) # just to get the right size

for i in range(sim_interval.size): 
        pos_ed  = combined_erodep[i, :, :] # get final one for comparision
        pred_erodep[i,:] = pos_ed.mean(axis=0)   

rmse, rmse_sed= mean_sqerror(  pred_erodep, pred_elev,  groundtruth_elev,  groundtruth_erodep_pts)


print ('time taken  in minutes = ', (timer_end-timer_start)/60)
print ('Folder: ', run_nb_str)
np.savetxt(fname+'/time_sqerror.txt',[ (timer_end-timer_start)/60,  rmse_sed, rmse], fmt='%1.2f'  )

#print(pos_param)

mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)

ax.boxplot(pos_param.T) 
ax.set_xlabel('Badlands parameters')
ax.set_ylabel('Posterior') 
plt.legend(loc='upper right') 
plt.title("Boxplot of Posterior")
plt.savefig(fname+'/badlands_pos.png')
plt.savefig(fname+'/badlands_pos.svg', format='svg', dpi=400)

print("Number of chains, problem, folder, time, RMSE_sed, RMSE")
print (num_chains, problemfolder, run_nb_str, (timer_end-timer_start)/60, rmse_sed, rmse)

# Check the output

In [ ]:
import matplotlib.image as mpimg

plt.figure(figsize=(12, 16))
img=mpimg.imread('crater/results_0/pos_distri_0_pos_.png')
imgplot = plt.imshow(img)
plt.show()

plt.figure(figsize=(12, 16))
img=mpimg.imread('crater/results_0/x_ymid_opt.png')
imgplot = plt.imshow(img)
plt.show()